In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Load dataset into a DataFrame
df = pd.read_csv("Walmart-Data-Analysis.csv")

# Display first few rows
df.head()

Saving Walmart-Data-Analysis.csv to Walmart-Data-Analysis.csv


,CustID,ProductID,Uniq Id,Crawl Timestamp,Product Url,Product Name,Description,List Price,Sale Price,Brand,...,Available,Label,Date,name,location,Rating,review,Image_Links,Past_Purchases,Clicks
0,1,1,51b010b871cde349bd32159a1cc1a15f,2020-01-24 16:08:36 +0000,https://www.walmart.com/ip/Allegiance-Economy-...,Allegiance Economy Dual-scale Digital Thermometer,We aim to show you accurate product informati...,11.11,11.11,Cardinal Health,...,True,Rollback,2023-04-25,A.,"Ankeny, IA",4,While it may be difficult to not shop at Walma...,https://example.com/image1.jpg,4,4
1,2,2,d6a7f100e44a626a3701804e99236ad6,2020-01-24 15:54:21 +0000,https://www.walmart.com/ip/Kenneth-Cole-Reacti...,Kenneth Cole Reaction Eau De Parfum Spray For ...,We aim to show you accurate product informati...,23.99,23.99,Kenneth Cole,...,True,Rollback,2022-08-16,DONNA,"Phoenix, AZ",2,While it may be difficult to not shop at Walma...,https://example.com/image1.jpg,1,9
2,3,3,99d2b7da7e3e427a942f864937dacd9d,2020-01-24 18:34:28 +0000,https://www.walmart.com/ip/Kid-Tough-Fitness-I...,Kid Tough Fitness Inflatable Free-Standing Pun...,We aim to show you accurate product informati...,30.76,30.76,BONK FIT,...,True,Rollback,2023-12-09,Jill,"Baton Rouge, LA",2,I have spent hours on the phone with no resolu...,https://example.com/image4.jpg,0,3
3,4,4,4c76d170c2c6a759cbce812d790a0b88,2020-01-24 11:08:53 +0000,https://www.walmart.com/ip/THE-FIRST-YEARS/167...,THE FIRST YEARS,We aim to show you accurate product informati...,6.99,6.99,The First Years,...,True,On Sale,2022-05-04,Sukanya,"Maumee, OH",1,I updated my phone number and I was trying to ...,https://example.com/image4.jpg,3,1
4,5,5,8ac95837dc8baa01e504fd8f633ffaf2,2020-03-10 07:37:21 +0000,https://www.walmart.com/ip/4-Pack-MD-USA-Seaml...,4 Pack - MD USA Seamless Toe-Wave-In Mesh Diab...,We aim to show you accurate product informatio...,28.27,28.27,MD USA,...,True,Rollback,2023-01-13,Tiffany,"Laurinburg, NC",1,I bought a 75 inch tv online using Walmart. Ri...,https://example.com/image4.jpg,3,5


In [ ]:
import os
os.listdir()


['.config', 'Walmart-Data-Analysis.csv', 'sample_data']

In [ ]:
file_path = "Walmart-Data-Analysis.csv"  

In [ ]:
!pip install gradio

In [ ]:
!pip install scikit-surprise

In [ ]:
file_path = "Walmart-Data-Analysis.csv"
df = pd.read_csv(file_path)

# Display sample user IDs
print(df["CustID"].unique()[:10])

[ 1  2  3  4  5  6  7  8  9 10]


In [ ]:
# Install necessary package
!pip install gradio surprise pandas numpy scikit-learn --quiet

In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, NMF, KNNBasic
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.accuracy import rmse
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split as sk_split
from sklearn.metrics import mean_squared_error

# Load dataset
file_path = "Walmart-Data-Analysis.csv"
df = pd.read_csv(file_path)

# Rename columns to standard names
df.rename(columns={"CustID": "userID", "ProductID": "itemID", "Sale Price": "rating"}, inplace=True)

# Convert Sale Price to numeric & drop missing
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df = df.dropna(subset=["userID", "itemID", "rating"])

# 🔥 Min-Max Scaling to normalize Sale Price
scaler = MinMaxScaler()
df["rating"] = scaler.fit_transform(df[["rating"]])

# Define Surprise Reader
reader = Reader(rating_scale=(0, 1))  # Since we normalized ratings between 0 and 1

# Load dataset into Surprise format
data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2)

# 🔥 Hyperparameter Tuning for SVD
param_grid = {
    "n_factors": [50, 100, 150],
    "lr_all": [0.002, 0.005, 0.01],
    "reg_all": [0.02, 0.1, 0.4]
}

gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3, n_jobs=-1)
gs.fit(data)

# Use Best SVD Model
svd = gs.best_estimator["rmse"]
svd.fit(trainset)
predictions_svd = svd.test(testset)
rmse_svd = rmse(predictions_svd)

# 🔥 KNN with Cosine Similarity
knn = KNNBasic(sim_options={"name": "cosine", "user_based": True})
knn.fit(trainset)
predictions_knn = knn.test(testset)
rmse_knn = rmse(predictions_knn)

# 🔥 NMF Model
nmf = NMF()
nmf.fit(trainset)
predictions_nmf = nmf.test(testset)
rmse_nmf = rmse(predictions_nmf)

# 🔥 XGBoost Predictive Model
df_xgb = df[["userID", "itemID", "rating"]]
X = df_xgb[["userID", "itemID"]]
y = df_xgb["rating"]

X_train, X_test, y_train, y_test = sk_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred))

# 🔥 Print Model Evaluation Results
results = {
    "SVD (Tuned)": rmse_svd,
    "KNN (Cosine)": rmse_knn,
    "NMF": rmse_nmf,
    "XGBoost": rmse_xgb
}

print("📊 Model Evaluation Results (Lower RMSE is Better)")
for model, rmse_value in results.items():
    print(f"🔹 {model} RMSE: {rmse_value:.4f}")

# Determine Best Model
best_model = min(results, key=results.get)

print(f"\n🏆 Best Performing Model: {best_model}")

RMSE: 0.0412
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.0412
RMSE: 0.0412
📊 Model Evaluation Results (Lower RMSE is Better)
🔹 SVD (Tuned) RMSE: 0.0412
🔹 KNN (Cosine) RMSE: 0.0412
🔹 NMF RMSE: 0.0412
🔹 XGBoost RMSE: 0.0757

🏆 Best Performing Model: SVD (Tuned)


In [ ]:
import pandas as pd
import gradio as gr
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Load dataset
file_path = "Walmart-Data-Analysis.csv"
df = pd.read_csv(file_path)

# Extract product categories
df['product_category'] = df['Category'].apply(lambda x: x.split('|')[0].strip() if pd.notna(x) else 'Unknown')
categories = df['product_category'].unique().tolist()

# Checking if necessary columns exist
required_columns = {'CustID', 'Product Name', 'Sale Price', 'Label', 'Rating'}
missing_columns = required_columns - set(df.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Convert Sale Price to numeric
df['Sale Price'] = pd.to_numeric(df['Sale Price'], errors='coerce').fillna(0)

# Filter rollback items only
rollback_items = df[df['Label'].str.lower() == 'rollback']

# Collaborative Filtering Model (SVD)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['CustID', 'ProductID', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
model = SVD()
model.fit(trainset)

def get_recommendations(user_id):
    """ Get Top 5 Recommended Products for the User """
    if user_id not in df['CustID'].values:
        return [["Invalid User ID", ""]]

    user_products = df[df['CustID'] == user_id]['ProductID'].unique()
    all_products = df['ProductID'].unique()

    # Predict ratings for products the user hasn't interacted with
    predictions = [(pid, model.predict(user_id, pid).est) for pid in all_products if pid not in user_products]

    # Sort by highest predicted rating
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_products = [df[df['ProductID'] == pid]['Product Name'].values[0] for pid, _ in predictions[:5]]

    return [[p, "Recommended"] for p in top_products]

def display_products(category):
    """ Returns 10 product names for the selected category """
    filtered_items = df[df['product_category'] == category].head(10)

    if filtered_items.empty:
        return [], "No products available in this category"

    product_names = filtered_items['Product Name'].tolist()

    return gr.update(choices=product_names), ""

def update_cart(selected_products, user_id):
    """ Updates the cart total, price gap, and recommendations based on rollback items and sentiment analysis """
    if not selected_products or user_id not in df['CustID'].values:
        return 0, 35, [["No rollback items match", ""]], [["Invalid User ID or No Products Selected", ""]]

    selected_items = df[df['Product Name'].isin(selected_products)]
    cart_total = selected_items['Sale Price'].sum()

    # Compute price gap
    free_delivery_threshold = 35
    price_gap = max(0, free_delivery_threshold - cart_total)

    # Get recommendations based on rollback items matching price gap
    matching_rollback_items = rollback_items[
        (rollback_items['Sale Price'] >= (price_gap - 5)) &
        (rollback_items['Sale Price'] <= (price_gap + 5))
    ]

    rollback_recommendations = matching_rollback_items[['Product Name', 'Sale Price']].head(5).values.tolist()

    # Get AI recommendations for user
    ai_recommendations = get_recommendations(user_id)

    return cart_total, price_gap, rollback_recommendations if rollback_recommendations else [["No rollback items match", ""]], ai_recommendations

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🛒 Personalized Cart Summary")

    with gr.Row():
        # Left Column: User ID & Product Selection
        with gr.Column():
            gr.Markdown("### 🔑 Enter User ID")
            user_id_input = gr.Number(label="User ID", interactive=True)

            gr.Markdown("### 📌 Select a Product Category")
            category_dropdown = gr.Dropdown(choices=categories, label="Product Categories", interactive=True)
            product_checkbox = gr.CheckboxGroup(label="Select Products", interactive=True)
            no_products_message = gr.Markdown("")

        # Right Column: Cart Summary & Recommendations
        with gr.Column():
            gr.Markdown("### 📊 Cart Details")
            total_display = gr.Textbox(label="Total ($)", interactive=False)
            price_gap_display = gr.Textbox(label="Price Gap ($)", interactive=False)

            gr.Markdown("### 🔥 Recommended Rollback Items")
            rollback_recommendation_display = gr.Dataframe(headers=["Product", "Price"], label="Rollback Recommendations")

            gr.Markdown("### 🤖 AI-Based Recommendations")
            ai_recommendation_display = gr.Dataframe(headers=["Product", "Status"], label="AI Recommendations")

    # Callbacks
    category_dropdown.change(display_products, category_dropdown, [product_checkbox, no_products_message])
    product_checkbox.change(update_cart, [product_checkbox, user_id_input],
                            [total_display, price_gap_display, rollback_recommendation_display, ai_recommendation_display])

# Launch the app
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2730530a503f5c311d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
